In [2]:
###
import csv
import numpy as np
from collections import defaultdict
import matplotlib.pyplot as plt
from pygsp import *
from scipy.spatial import distance
import mplleaflet
import networkx as nx
import branca.colormap as cm
#####################################
columns_signal = defaultdict(list) 

with open('PE_pluviometria_sbrt2.csv') as f:
    reader = csv.reader(f)
    next(reader)
    for row in reader:
        for (i,v) in enumerate(row):
            columns_signal[i].append(v)
            

l = len(columns_signal[0])

columns_coordinates = defaultdict(list) 

with open('Coordenadas_das_estacoes_sbrt2.csv') as f:
    reader = csv.reader(f)
    next(reader)
    for row in reader:
        for (i,v) in enumerate(row):
            columns_coordinates[i].append(v)
            

k = len(columns_coordinates[0])
#xp=0
#####################################
Data_matrix = np.zeros((l,3))

for j in range (0,l):
    if (columns_signal[3][j] != '-'):
        Data_matrix[j][0] = float(columns_coordinates[2][j])
        Data_matrix[j][1] = float(columns_coordinates[3][j])
        Data_matrix[j][2] = float(columns_signal[3][j])
    else:
        Data_matrix[j][0] = float(columns_coordinates[2][j])
        Data_matrix[j][1] = float(columns_coordinates[3][j])
        Data_matrix[j][2] = -1
        #print(xp)
        #xp=xp+1

coordinates = np.zeros((len(Data_matrix),2))
signal = np.zeros((len(Data_matrix),1))

for i in range(0,len(Data_matrix)):
    coordinates[i][1] = Data_matrix[i][0]
    coordinates[i][0] = Data_matrix[i][1]
    signal[i] = Data_matrix[i][2]
    
#####################################

def m_adj(coords,k):
    
    Adj_K = np.zeros((len(coords),len(coords)))
    Aux = np.zeros((len(coords),1))
    aux = 0

    ind=np.zeros((k,1))
    for x in range(0,len(coords)):
        for y in range(0,len(coords)):
            v = [coords[x][0],coords[x][1]]
            w = [coords[y][0],coords[y][1]]
            dst = distance.euclidean(v,w)
            Aux[y]=dst
        ind = sorted(Aux)[1:k+1]
        for i in range(0,k):
            ind[i] = np.where(Aux == ind[i])
        for j in range(0,k):
            aux = ind[j][0]
            Adj_K[x][int(aux)] = np.exp((-Aux[ind[j]]**2))
            Adj_K[int(aux)][x] = Adj_K[x][int(aux)]

    
    return(Adj_K)

A = m_adj(coordinates,4)

#####################################
G=nx.Graph()

lst = []

for i in range(0,len(signal)):
    lst.append(i)

G.add_nodes_from(lst)

#####################################
pos = []

for i in range(0,len(signal)):
    pos.append((coordinates[i][0],coordinates[i][1]))

edges = []

for i in range(0,len(A)):
    for j in range(0,len(A)):
        if A[i][j] > 0:
            edges.append((i,j))

pos_edges = []

for i in range(0,len(A)):
    pos_edges.append(((coordinates[edges[i][0]],coordinates[edges[i][0]]),(coordinates[edges[i][1]],coordinates[edges[i][1]])))
            
G.add_edges_from(edges)

#####################################
color = []

a = cm.linear.RdBu_04.scale(0, 280)

for i in range(0,len(coordinates)):
    if signal[i] > 0:
        color.append(a.rgb_hex_str(signal[i]))
    else:
        color.append('#000000')

#####################################
fig, ax = plt.subplots()

nx.draw_networkx_nodes(G,pos=pos,node_size=45,node_color=color,edge_color='k',alpha=1, with_labels=True)
nx.draw_networkx_edges(G,pos=pos,edge_color='black', alpha=.5)
nx.draw_networkx_labels(G,pos, label_pos =10.3)

mplleaflet.display(fig=ax.figure)
#mplleaflet.show(fig=ax.figure)

/home/default/.local/lib/python3.5/site-packages/networkx/drawing/nx_pylab.py:611: MatplotlibDeprecationWarning: isinstance(..., numbers.Number)
  if cb.is_numlike(alpha):
